In [2]:
# Based on this paper - https://arxiv.org/pdf/1706.03762.pdf
# Might want to move layer norm inside the residual block - https://arxiv.org/pdf/2002.04745.pdf
# Layer normalization - https://arxiv.org/pdf/1607.06450.pdf
#!pip install torch torchtext sentencepiece datasets

In [3]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 5000
TRAIN_SIZE = 5000

In [4]:
from text_data import Opus100DatasetWrapper

class Wrapper(Opus100DatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = 40
    target_length = 40

wrapper = Wrapper(SP_VOCAB_SIZE)
datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
valid = datasets["validation"]

Found cached dataset opus100 (/Users/vik/.cache/huggingface/datasets/opus100/en-es/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
100%|██████████| 3/3 [00:00<00:00, 21.94it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=opus100 --vocab_size=5000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: opus100
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extre

In [19]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_units, attention_heads, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.head_units = int(input_units/attention_heads)
        self.mask = mask
        if self.head_units * self.attention_heads != self.input_units:
            raise Exception("Invalid input units and heads combo")

        k = math.sqrt(1/self.head_units)
        self.query_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)
        self.key_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)
        self.value_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)

    def forward(self, queries, context):
        # convert to 4d tensor with batch_size, attn_heads, seq_len, embedding_dim
        exp_queries = queries.unsqueeze(1).expand(-1, self.attention_heads, -1,-1)
        exp_context = context.unsqueeze(1).expand(-1, self.attention_heads, -1,-1)
        proj_queries = torch.einsum("base, aeh->bash", exp_queries, self.query_weights)
        proj_keys = torch.einsum("base, aeh->bash", exp_context, self.key_weights)
        proj_values = torch.einsum("base, aeh->bash", exp_context, self.value_weights)

        attention = torch.einsum("bash, bahk->bask", proj_queries, torch.transpose(proj_keys, -2, -1)) / np.sqrt(self.head_units)
        if self.mask:
            # Prevent decoder queries from looking at tokens that come after
            # Do this by setting attention to negative infinity, so it is softmaxed to zero in the next step
            mask = torch.zeros((attention.shape[-2], attention.shape[-1]), device=DEVICE)
            mask_indices = np.triu_indices(attention.shape[-2], k=1, m=attention.shape[-1])
            mask[mask_indices] = -torch.inf
            attention += mask

        # Softmax on last dimension
        # Sequence-wise softmax, so attention between one sequence and other sequences sums to 1
        attention = torch.softmax(attention, dim=-1)
        weighted_values = torch.einsum("bash, bahe->base", attention, proj_values)

        # Swap attention head and sequence axis, then reshape to batch, seq, embedding
        weighted_values = weighted_values.swapaxes(1,2).reshape(queries.shape[0], queries.shape[1], -1)
        return weighted_values

In [20]:
class EncoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048):
        super(EncoderBlock, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.hidden_units = hidden_units

        self.mha = MultiHeadAttention(self.input_units, self.attention_heads)
        self.dropouts = nn.ModuleList(nn.Dropout(.1) for _ in range(2))
        self.linear1 = nn.Linear(self.input_units, hidden_units)
        self.linear2 = nn.Linear(hidden_units, self.input_units)
        self.relu = nn.ReLU()
        self.lns = nn.ModuleList(nn.LayerNorm(self.input_units) for _ in range(2))

    def forward(self, x):
        weighted_values = self.dropouts[0](self.mha(x, x))
        attn_output = self.lns[0](x + weighted_values)

        reprojected = self.dropouts[1](self.linear2(self.relu(self.linear1(attn_output))))
        block_output = self.lns[1](attn_output + reprojected)
        return block_output

In [21]:
class DecoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048):
        super(DecoderBlock, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.hidden_units = hidden_units

        self.in_attn = MultiHeadAttention(self.input_units, self.attention_heads, mask=True)
        self.context_attn = MultiHeadAttention(self.input_units, self.attention_heads)
        self.dropouts = nn.ModuleList([nn.Dropout(.1) for _ in range(3)])
        self.linear1 = nn.Linear(self.input_units, hidden_units)
        self.linear2 = nn.Linear(hidden_units, self.input_units)
        self.relu = nn.ReLU()
        self.lns = nn.ModuleList(nn.LayerNorm(self.input_units) for _ in range(3))

    def forward(self, queries, context):
        weighted_values = self.dropouts[0](self.in_attn(queries, queries))
        attn_output = self.lns[0](queries + weighted_values)

        decoder_values = self.dropouts[1](self.context_attn(attn_output, context))
        decoder_output = self.lns[1](attn_output + decoder_values)

        reprojected = self.dropouts[2](self.linear2(self.relu(self.linear1(decoder_output))))
        block_output = self.lns[2](decoder_output + reprojected)
        return block_output

In [22]:
class Transformer(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, enc_sequence_len, blocks=1):
        super(Transformer, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.attention_heads = attention_heads
        self.blocks = blocks

        k = math.sqrt(1/self.hidden_units)
        self.output_embedding = nn.Parameter(torch.rand(hidden_units, input_units) * 2 * k - k)
        self.embedding = nn.Embedding(input_units, hidden_units)
        self.dropouts = nn.ModuleList(nn.Dropout(.1) for _ in range(2))
        self.encoders = nn.ModuleList(EncoderBlock(hidden_units, attention_heads) for _ in range(self.blocks))
        self.decoders = nn.ModuleList(DecoderBlock(hidden_units, attention_heads) for _ in range(self.blocks))
        self.pos_encoding = self.encoding(enc_sequence_len, self.hidden_units).to(DEVICE)


    def embed(self, indices, reverse=False):
        if reverse:
            return torch.einsum("bse, eo->bso", indices, self.embedding.T)
        else:
            one_hot = F.one_hot(indices.to(torch.long), num_classes=self.input_units).to(torch.float)
            return torch.einsum("bso, oe->bse", one_hot, self.embedding)

    def forward(self, x, y, enc_outputs=None):
        if enc_outputs is None:
            # 3D with batch, seq, embeddings
            # TODO: Tie input and output embedding weights
            enc_outputs = self.dropouts[0](self.embedding(x) + self.pos_encoding[:x.shape[1]])

            for i in range(self.blocks):
                enc_outputs = self.encoders[i](enc_outputs)

        dec_outputs = self.dropouts[1](self.embedding(y) + self.pos_encoding[:y.shape[1]])
        for i in range(self.blocks):
            dec_outputs = self.decoders[i](dec_outputs, enc_outputs)

        token_vectors = torch.einsum("bse, eo->bso", dec_outputs, self.output_embedding)
        return token_vectors, enc_outputs

    def encoding(self, seq_len, embed_len):
        encodings = torch.zeros((seq_len, embed_len))
        for i in range(seq_len):
            all = torch.exp(torch.arange(0, embed_len, 2) * (-math.log(10000.0) / embed_len))
            encodings[i, 0::2] = torch.sin(i * all)
            encodings[i, 1::2] = torch.cos(i * all)
        return encodings

In [23]:
def generate(sequence, pred, target, wrapper):
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(target.cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [24]:
from tqdm.auto import tqdm

# TODO: Profile and improve perf - https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html
model = Transformer(wrapper.vocab_size, 512, 8, wrapper.y_length, blocks=1).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=wrapper.pad_token)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
EPOCHS = 100
DISPLAY_BATCHES = 2
OUT_SEQUENCE_LEN = wrapper.y_length
PRINT_VALID = True

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad(set_to_none=True)
        pred, _ = model(sequence.to(DEVICE), prev_target.to(DEVICE))

        # If you use a batch, need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        print(f"Epoch {epoch} train loss: {train_loss / len(train) / BATCH_SIZE}")
        sents = generate(sequence, pred, target, wrapper)
        for sent in sents[:DISPLAY_BATCHES]:
            print(sent)

        if PRINT_VALID and epoch % 10 ==0:
            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            # Deactivate dropout layers
            model.eval()
            for batch, (sequence, target, prev_target) in tqdm(enumerate(valid)):
                # Inference token by tokens
                sequence = sequence.to(DEVICE)
                outputs = prev_target[:,0].unsqueeze(1).to(DEVICE)
                enc_outputs = None
                # TODO: Investigate memory leak with valid generation
                for i in range(OUT_SEQUENCE_LEN):
                    pred, enc_outputs = model(sequence, outputs, enc_outputs=enc_outputs)
                    last_output = torch.argmax(pred, dim=2)
                    outputs = torch.cat((outputs, last_output[:,-1:]), dim=1)
                loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
                valid_loss += loss.item()
            print(f"Valid loss: {valid_loss / len(valid) / BATCH_SIZE}")
            sents = generate(sequence, pred, target, wrapper)
            for sent in sents[:DISPLAY_BATCHES]:
                print(sent)
            # Reactivate dropout
            model.train()

127it [00:24,  5.24it/s]


Epoch 0 train loss: 0.1742700781878524
¿De dónde la conoces? | How do you know her? | You you you you you?
siguiente: paisaje de un doble planeta enano | next: landscape of a double dwarf planet | The'


15it [00:12,  1.24it/s]


Valid loss: 0.21981802781422932
También son de parte de Jesús. | They're from Jesus too. | You't you't you't you't you't you't you.
Está tan grande. | He's so big. | You't't't.


127it [00:24,  5.08it/s]


...

127it [00:24,  5.09it/s]


Epoch 3 train loss: 0.1220014655566591
Sin embargo, he determinado su naturaleza. | However, I've determined its nature. | Youever, I'm been....
¿Por qué me enganchaste con esa gorda pelada? - ¿Quién? ¿Rita? | Rita? | - withing


127it [00:25,  5.08it/s]


Epoch 4 train loss: 0.10627326193287617
La compañía desarrolla sus actividades como una compañía de danza contemporánea internacional con base en Barcelona. | The company developed its activities as an international contemporary dance company, based in Barcelona. | It amount develops you considera of a internationalcegsma .... but. the.
Nunca nos separaremos otra vez. | We shall never again be parted. | I' be not.ly..


118it [00:23,  5.02it/s]

In [ ]:
from torchinfo import summary

print(summary(model))

In [18]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, ) as prof:
    model(sequence.to(DEVICE), prev_target.to(DEVICE))

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

STAGE:2023-01-21 20:45:14 90620:35032007 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-21 20:45:14 90620:35032007 ActivityProfilerController.cpp:300] Completed Stage: Collection


---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
               aten::einsum         0.52%     345.000us        41.20%      27.273ms       1.705ms            16  
                  aten::bmm        36.82%      24.372ms        37.10%      24.561ms       1.535ms            16  
              aten::dropout         0.16%     104.000us        32.51%      21.523ms       3.075ms             7  
           aten::bernoulli_        28.30%      18.732ms        28.30%      18.732ms       2.676ms             7  
               aten::linear         0.09%      58.000us        16.26%      10.761ms       2.690ms             4  
                aten::addmm        14.37%       9.514ms        16.01%      10.600ms     